In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# 파일 불러오기
sheet_name = "input_test"
input_path = os.path.join(os.getcwd(), "input\\")
input_test = pd.read_excel(input_path+"test_test.xlsx", sheet_name=sheet_name, header=0)
print(input_test.shape)
input_test.head(5)

(1364, 14)


,No,용어명,용어명 분리,용어설명,영문명,영문약어명,도메인명,데이터타입,데이터길이,소수점이하,생성일(제정일),범정부표준여부,정보시스템(DB),관련필드/용어
0,1,가입일자,가입_일자,"조직이나 단체 따위에 들어가거나, 서비스를 제공하는 상품 따위를 신청한 날짜를 연월...",NaN,NaN,연월일C8,CHAR,8,NaN,2020-08-13,범정부표준,NaN,NaN
1,2,개설일자,개설_일자,설비나 제도 따위를 새로 마련하고 그에 관한 일을 처음 시작한 날짜를 연월일로 표현,NaN,NaN,연월일C8,CHAR,8,NaN,2020-08-13,범정부표준,NaN,NaN
2,3,개시일자,개시_일자,시장을 처음 열어 물건의 매매를 시작한 날짜를 연월일로 표현,NaN,NaN,연월일C8,CHAR,8,NaN,2020-08-13,범정부표준,NaN,NaN
3,4,개업일자,개업_일자,영업을 처음 시작한 날짜를 연월일로 표현,NaN,NaN,연월일C8,CHAR,8,NaN,2020-08-13,범정부표준,NaN,NaN
4,5,개인정보 동의여부,개인_정보_동의_여부,특정 개인을 식별하거나 유추해 낼 수 있는 모든 정보를 공동 이용하도록 허락할지 참...,NaN,NaN,여부C1,CHAR,1,NaN,2020-08-13,범정부표준,NaN,NaN


In [3]:
# 용어명 분리
# 스페이스 제거
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")

# 인덱스 컬럼 추가
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']

# 용어명 분리
input_split_kor = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())

# 영문명 없을 경우 대비하여 cnt 집계
kor_firts_cnt = len(input_split_kor.columns)

# 컬럼명 입력
name_list_kor = []
for i in range(len(input_split_kor.columns)):
    name_list_kor.append("stwd_kor{}".format(i))
input_split_kor.columns = name_list_kor

# 한글 개수 세기
dupl_list_kor = []
dupl_cnt_kor = []
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = []
    for j in range(len(input_split_kor["stwd_kor{}".format(i)])):
        dupl_list_kor = input_split_kor["stwd_kor{}".format(i)].to_list().count(input_split_kor["stwd_kor{}".format(i)][j])
        globals()["dupl_list_kor{}".format(i)].append(dupl_list_kor)
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_kor{}".format(i)])], axis = 0)
    globals()["dupl_list_kor{}".format(i)].columns = ["dupl_list_kor{}".format(i)]
    
# 컬럼명,index, 단어위치 추가
for i in range(len(input_split_kor.columns)):
    input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
    globals()["stwd_kor{}".format(i)] = pd.concat(
            [
            df_index, 
            pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]),
            pd.DataFrame(input_split_kor["num_kor{}".format(i)]),
            globals()["dupl_list_kor{}".format(i)]
            ], 
            axis=1)
print(name_list_kor)
print(kor_firts_cnt)
stwd_kor0
# dupl_list_kor0

['stwd_kor0', 'stwd_kor1', 'stwd_kor2', 'stwd_kor3', 'stwd_kor4', 'stwd_kor5', 'stwd_kor6', 'stwd_kor7', 'stwd_kor8', 'stwd_kor9']
10


,index,stwd_kor0,num_kor0,dupl_list_kor0
0,1,가입,num_kor0,1
1,2,개설,num_kor0,1
2,3,개시,num_kor0,1
3,4,개업,num_kor0,1
4,5,개인,num_kor0,1
...,...,...,...,...
1359,1360,정액,num_kor0,3
1360,1361,할인,num_kor0,13
1361,1362,운임,num_kor0,20
1362,1363,운임,num_kor0,20


In [4]:
# 영문명 분리
# 스페이스 제거
if (input_test['영문약어명'].isnull()).sum() == 0:
    input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")

    # 인덱스 컬럼 추가
    df_index = pd.DataFrame(input_test['No'])
    df_index.columns = ['index']

    # 영문명 분리
    input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
    
    # 한글명 없을 경우 대비하여 cnt 집계
    eng_firts_cnt = len(input_split_eng.columns)
    
    # 컬럼명 입력
    name_list_eng = []
    for i in range(len(input_split_eng.columns)):
        name_list_eng.append("stwd_eng{}".format(i))
    input_split_eng.columns = name_list_eng
    
    # 영문 개수 세기
    dupl_list_eng = []
    dupl_cnt_eng = []
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = []
        for j in range(len(input_split_eng["stwd_eng{}".format(i)])):
            dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
            globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
        globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]
        
    # 컬럼명,index, 단어위치 추가
    for i in range(len(input_split_eng.columns)):
        input_split_eng["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)] = pd.concat(
                [
                df_index, 
                pd.DataFrame(input_split_eng["stwd_eng{}".format(i)]),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)]),
                globals()["dupl_list_eng{}".format(i)]
                ], 
                axis=1)
else:
    name_list_eng = []
    for i in range(kor_firts_cnt):
        globals()["stwd_eng{}".format(i)] = pd.DataFrame(columns = ["stwd_eng{}".format(i)])
        globals()["num_eng{}".format(i)] = pd.DataFrame(columns = ["num_eng{}".format(i)])
        globals()["dupl_list_eng{}".format(i)] = pd.DataFrame(columns = ["dupl_list_eng{}".format(i)])
        globals()["stwd_eng{}".format(i)] = pd.concat(
            [
                df_index , 
                globals()["stwd_eng{}".format(i)] ,
                globals()["num_eng{}".format(i)],
                globals()["dupl_list_eng{}".format(i)],
            ]
            , axis = 1
        )
        globals()["stwd_eng{}".format(i)]["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)]["dupl_list_eng{}".format(i)] = len(globals()["stwd_eng{}".format(i)])
        name_list_eng.append("stwd_eng{}".format(i))

# 컬럼수 일치 점검
if len(name_list_kor) == len(name_list_eng):
    print("한글컬럼수와 영문컬럼수 일치")
else:
    print("한글컬럼수와 영문컬럼수 불일치")
                                                       
print(name_list_eng)
stwd_eng5

한글컬럼수와 영문컬럼수 일치
['stwd_eng0', 'stwd_eng1', 'stwd_eng2', 'stwd_eng3', 'stwd_eng4', 'stwd_eng5', 'stwd_eng6', 'stwd_eng7', 'stwd_eng8', 'stwd_eng9']


,index,stwd_eng5,num_eng5,dupl_list_eng5
0,1,NaN,num_eng5,1364
1,2,NaN,num_eng5,1364
2,3,NaN,num_eng5,1364
3,4,NaN,num_eng5,1364
4,5,NaN,num_eng5,1364
...,...,...,...,...
1359,1360,NaN,num_eng5,1364
1360,1361,NaN,num_eng5,1364
1361,1362,NaN,num_eng5,1364
1362,1363,NaN,num_eng5,1364


In [5]:
# 한글/영문 컬럼 수 일치시키기
name_cnt_kor = len(name_list_kor)
name_cnt_eng = len(name_list_eng)
if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
    print('한글컬럼과 영문컬럼수 일치')
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)]),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)])
            ],
            axis = 1
        )
    print('한글컬럼이 더 많은 싱태')
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    for i in name_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)]),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)])
            ],
            axis = 1
        )
    print('영문컬럼이 더 많은 싱태')       

stwd_eng7

한글컬럼과 영문컬럼수 일치


,index,stwd_eng7,num_eng7,dupl_list_eng7
0,1,NaN,num_eng7,1364
1,2,NaN,num_eng7,1364
2,3,NaN,num_eng7,1364
3,4,NaN,num_eng7,1364
4,5,NaN,num_eng7,1364
...,...,...,...,...
1359,1360,NaN,num_eng7,1364
1360,1361,NaN,num_eng7,1364
1361,1362,NaN,num_eng7,1364
1362,1363,NaN,num_eng7,1364


In [6]:
# left join
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )

# 카운트 일치 여부 점검
index_cnt = len(df_index['index'])
if index_cnt != len(stwd_total0):
    print("index개수 와 joined_date개수 불일치")
else:
    print("row count 일치")
stwd_total7

row count 일치


,index,stwd_kor7,num_kor7,dupl_list_kor7,stwd_eng7,num_eng7,dupl_list_eng7
0,1,None,num_kor7,1355,NaN,num_eng7,1364
1,2,None,num_kor7,1355,NaN,num_eng7,1364
2,3,None,num_kor7,1355,NaN,num_eng7,1364
3,4,None,num_kor7,1355,NaN,num_eng7,1364
4,5,None,num_kor7,1355,NaN,num_eng7,1364
...,...,...,...,...,...,...,...
1359,1360,None,num_kor7,1355,NaN,num_eng7,1364
1360,1361,None,num_kor7,1355,NaN,num_eng7,1364
1361,1362,None,num_kor7,1355,NaN,num_eng7,1364
1362,1363,None,num_kor7,1355,NaN,num_eng7,1364


In [7]:
# stwd_total 힌즐 세우기
df_stwd_total = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    globals()["df_stwd_eng{}".format(i)] = input_split_eng["stwd_eng{}".format(i)]
    df_stwd_eng = pd.concat([df_stwd_eng, globals()["df_stwd_eng{}".format(i)]])
df_stwd_eng.columns = ['stwd_eng']
df_stwd_eng

NameError: name 'input_split_eng' is not defined

In [ ]:
############ num_kor(n) / num_eng(n) 기준으로 테이블 분리
############ index 기준으로 left join하여 원상복구

In [ ]:
# left join
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )

# 카운트 일치 여부 점검
index_cnt = len(df_index['index'])
if index_cnt != len(stwd_total0):
    print("index 와 joined_date count불일치")
else:
    print("count 일치")
stwd_total5

In [ ]:
# merge
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.concat(
        [
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)]
        ],
        axis = 1        
    )
    print(i)
# print(name_cnt)
# print(name_cnt_diff_list)


stwd_total1

In [ ]:
for i in list(set(range(name_cnt)) - set(range(name_cnt_diff))):
    print(i)


In [ ]:
list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))

In [ ]:
input_split_eng

In [ ]:
input_split_eng['stwd_eng6'] = None
input_split_eng['stwd_eng6']

In [ ]:
input_split_eng['stwd_eng6']

In [ ]:
# 한글/영문 컬럼 수 비교하기
name_cnt_kor = len(name_list_kor)
name_cnt_eng = len(name_list_eng)

if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)]),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)])
            ],
            axis = 1
        )
        
        
        
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    for i in name_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)]),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)])
            ],
            axis = 1
        )
        

stwd_eng7


In [ ]:
stwd_eng6

In [ ]:
# 한글/영문 컬럼 수 비교하기
name_cnt_kor = len(name_list)
name_cnt_eng = len(name_list_eng)

if name_cnt_kor == name_cnt_eng:
    name_cnt = name_cnt_kor
    name_cnt_diff_list = None
elif name_cnt_kor > name_cnt_eng:
    name_cnt = name_cnt_eng
    name_cnt_diff_list = list(set(range(name_cnt_kor)) - set(range(name_cnt_eng)))
    for i in name_cnt_diff_list:
        globals()['stwd_eng{}'.format(i)] = None
elif name_cnt_kor < name_cnt_eng:
    name_cnt = name_cnt
    name_cnt_diff_list = list(set(range(name_cnt_eng)) - set(range(name_cnt_kor)))
    
    

    
    
for i in range(name_cnt):
    globals()['stwd_total{}'.format(i)] = pd.concat(
        [
        globals()['stwd{}'.format(i)],
        globals()['stwd_eng{}'.format(i)]
        ],
        axis = 1        
    )
    print(i)
# print(name_cnt)
# print(name_cnt_diff_list)

In [ ]:
# 리스트 - 리스트
list1 = [1,2]
list2 = [1,2,3]
list(set(list2) - set(list1))
list(set(list1) - set(list2))
list(set(list1).difference(set(list2)))
list(set(range(10)) - set(range(2)))

In [ ]:
list(set(range(10)) - set(range(2)))

In [ ]:
list(set(range(2)) - set(range(10)))

In [ ]:
# 용어명 분리
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
name_list = []
for i in range(len(input_split.columns)):
    name_list.append("stwd{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = name_list)
# 인덱스 컬럼 추가
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']
input_split= pd.concat([input_split, df_index], axis = 1)
print(input_split.shape)
input_split.head(5)

In [ ]:
# 영문명 분리
input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
name_list_eng = []
for i in range(len(input_split_eng.columns)):
    name_list_eng.append("stwd_eng{}".format(i))
print(name_list_eng)
# 컬럼명 만들기
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist(), columns = name_list_eng)
# 인덱스 컬럼 추가
# df_index = pd.DataFrame(input_test['No'])
# df_index.columns = ['index']
# input_split_eng= pd.concat([input_split_eng, df_index], axis = 1)
print(input_split_eng.shape)
input_split_eng.head(5)

In [ ]:
name_list = []
for i in range(len(input_split_eng.columns)):
    name_list_eng.append("stwd_eng{}".format(i))

In [ ]:
if len(name_list) == len(name_list_eng):
    name_list_cnt = len(name_list)
elif len(name_list) > len(name_list_eng):
    name_list_cnt = len(name_list)
elif len(name_list) < len(name_list_eng):
    name_list_cnt = len(name_list_eng)
    
for i in range(name_list_cnt):
    print(i)

In [ ]:
if len(name_list) == len(name_list_eng):
    print(1)
elif len(name_list) > len(name_list_eng):
    print(2)    
elif len(name_list) < len(name_list_eng):
    print(3)

In [ ]:
len(name_list_eng)

In [ ]:
len(name_list)

In [ ]:
# 용어명 분리
input_test['용어명 분리'] = input_test['용어명 분리'].str.replace(" ", "")
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
name_list = []
for i in range(len(input_split.columns)):
    name_list.append("stwd{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = name_list)
print(input_split.shape)
input_split.head(5)

In [ ]:
# 단어별 개수 세기(네이밍 컬럼)
dupl_list = []
dupl_cnt = []
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []
    for j in range(len(input_split["stwd{}".format(i)])):
        dupl_cnt = input_split["stwd{}".format(i)].to_list().count(input_split["stwd{}".format(i)][j])
        globals()["dupl_list{}".format(i)].append(dupl_cnt)
# df_dupl 힌즐 세우기
df_dupl = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)]
    df_dupl = pd.concat([df_dupl, pd.DataFrame(globals()["dupl_list{}".format(i)])], axis = 0)
df_dupl.columns = ['dupl']
df_dupl

In [ ]:
# df_index 한줄 세우기
# df_index = pd.DataFrame(input_test['No'])
df_index = pd.DataFrame()
for i in range(len(input_split.columns)):
#     globals()["df_index{}".format(i)] = pd.DataFrame(input_test['No'])
    df_index = pd.concat([df_index, pd.DataFrame(input_test['No'])])
df_index.columns = ['index']
df_index

In [ ]:
# df_stwd 힌즐 세우기
df_stwd = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["df_stwd{}".format(i)] = input_split["stwd{}".format(i)]
    df_stwd = pd.concat([df_stwd, globals()["df_stwd{}".format(i)]])
df_stwd.columns = ['stwd']
df_stwd

In [ ]:
# df_chk
df_chk = pd.concat([df_stwd, df_index], axis=1).reset_index() # 인덱스 초기화
df_chk = df_chk[['stwd', 'index']]
print(df_chk.shape)
df_chk.tail(5)

In [ ]:
### 한글 단어 분리

In [ ]:
# 영문명 분리
input_test['영문약어명'] = input_test['영문약어명'].str.replace(" ", "")
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist())
name_list = []
for i in range(len(input_split_eng.columns)):
    name_list.append("stwd_eng{}".format(i))
print(name_list)
# 컬럼명 만들기
input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist(), columns = name_list)
print(input_split_eng.shape)
input_split_eng.head(5)

In [ ]:
# df_index 한줄 세우기
df_index_eng = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    df_index_eng = pd.concat([df_index_eng, pd.DataFrame(input_test['No'])])
df_index_eng.columns = ['index']
df_index_eng

In [ ]:
# df_stwd_eng 힌즐 세우기
df_stwd_eng = pd.DataFrame()
for i in range(len(input_split_eng.columns)):
    globals()["df_stwd_eng{}".format(i)] = input_split_eng["stwd_eng{}".format(i)]
    df_stwd_eng = pd.concat([df_stwd_eng, globals()["df_stwd_eng{}".format(i)]])
df_stwd_eng.columns = ['stwd_eng']
df_stwd_eng

In [ ]:
# df_chk_eng
df_chk_eng = pd.concat([df_stwd_eng, df_index_eng], axis=1).reset_index() # 인덱스 초기화
df_chk_eng = df_chk_eng[['stwd_eng', 'index']]
print(df_chk_eng.shape)
df_chk_eng.tail(5)

In [ ]:
### 영문 단어 분리

In [ ]:
df_chk.merge(df_chk_eng, on='index' 

In [ ]:
# df_chk_uni
df_chk_uni = df_chk[df_chk['stwd'].duplicated()==False].reset_index()
df_chk_uni = df_chk_uni[['stwd', 'index']]
df_chk_uni.tail(5)

In [ ]:
# unique 단어 개수 세기
dupl_list = []
dupl_cnt = []
for i in range(len(df_chk_uni["stwd"])):
    dupl_cnt = df_chk_uni["stwd"].to_list().count(df_chk_uni["stwd"][i])
    dupl_list.append(dupl_cnt)

len(dupl_list)

In [ ]:
# 횟수 병합
df_chk_uni = pd.concat([df_chk_uni, pd.DataFrame(dupl_list)], axis = 1)
df_chk_uni.columns = ['stwd', 'index', 'dupl_cnt']
print(df_chk_uni[df_chk_uni['dupl_cnt'] != 1])
print(df_chk_uni[df_chk_uni['dupl_cnt'] == None])
df_chk_uni.tail(5)

In [ ]:
# 점검 , 변환
# ab['dupl_cnt'].info()
# ab['dupl_cnt'].astype(int)

In [ ]:
# df_chk_uni
df_chk_dupl = df_chk[df_chk['stwd'].duplicated()].reset_index()
df_chk_dupl = df_chk_dupl[['stwd', 'index']]
df_chk_dupl.tail(5)

In [ ]:
# dupl 단어 개수 세기
dupl_list = []
dupl_cnt = []
for i in range(len(df_chk_dupl["stwd"])):
    dupl_cnt = df_chk_dupl["stwd"].to_list().count(df_chk_dupl["stwd"][i])
    dupl_list.append(dupl_cnt)

len(dupl_list)

In [ ]:
# 횟수 병합
df_chk_dupl = pd.concat([df_chk_dupl, pd.DataFrame(dupl_list)], axis = 1)
df_chk_dupl.columns = ['stwd', 'index', 'dupl_cnt']
print(df_chk_dupl[df_chk_dupl['dupl_cnt'] == 1])
print(df_chk_dupl[df_chk_dupl['dupl_cnt'] == None])
df_chk_dupl.tail(5)

In [ ]:
df_chk_dupl[df_chk_dupl['stwd'] == '매수']

In [ ]:
cc = df_chk_dupl.merge(input_test, left_on = 'index', right_on = 'No', how='left')
cc = cc[cc['stwd'].duplicated()==False]
print(cc.shape)
cc.head(3)

In [ ]:
df_chk_dupl.to_excel(input_path+"dsfadas.xlsx")

In [ ]:
cc = df_chk_dupl['stwd'].drop_duplicates()
(cc.shape)
cc

In [ ]:
cc.to_excel(input_path+"cccccccccccc.xlsx")

In [ ]:
df_chk_dupl['stwd'].drop_duplicates()

In [ ]:
ccc = df_chk_dupl.merge(input_test, left_on = 'index', right_on = 'No', how='left')
ccc = ccc[ccc['stwd'].duplicated()]
print(ccc.shape)
ccc.head(3)

In [ ]:
ccc.to_excel(input_path+"cccccccccc3cc.xlsx")

In [ ]:
a[a['stwd'].duplicated()]

In [ ]:
a[pd.DataFrame(a['stwd']).duplicated(keep='first')]

In [ ]:
a['stwd'].drop_duplicates()

In [ ]:
a = pd.concat([df_stwd, df_dupl, df_index], axis=1)
a

In [ ]:
tOK = a.loc[a['dupl'] == 1]
# tOK['stwd'].dropna()
tOK[tOK['stwd'] == '검토']
tOK
tNO = a.loc[a['dupl'] > 1]
# tNO['stwd'].dropna()
tNO[tNO['stwd'] == '검토']
tNO

In [ ]:
tOK['stwd'].drop_duplicates()

In [ ]:
print(368 + 412)
tNO['stwd'].drop_duplicates()

In [ ]:
a['stwd'].drop_duplicates()

In [ ]:
t2['stwd'].drop_duplicates()

In [ ]:
a.to_excel(input_path+"\\dfdf.xlsx")

In [ ]:
tOK.to_excel(input_path+"\\dfdfOK.xlsx")

In [ ]:
tNO.to_excel(input_path+"\\dfdfNO.xlsx")

In [ ]:
tNO['stwd'].drop_duplicates().to_excel(input_path+"\\dfdfNO_DUP.xlsx")

In [ ]:
t2['stwd'].drop_duplicates().tail(30)

In [ ]:
# t2 = a[a['stwd'].duplicated(keep='first')]
t2 = a[a['stwd'].duplicated(False)]
t2[t2['stwd'] == '검토']
# t2['stwd'].dropna()
t2

In [ ]:
t2.drop_

In [ ]:
a[a['stwd'] == '검토']

In [ ]:
a.head(20)

In [ ]:
a['stwd'].duplicated(keep='first').head(20)
# t#['stwd'].dropna()
# t3[t3['stwd'] == '검토']

In [ ]:
# # concat 을 위한 DataFame 묶기 (o 시리즈여야 한곳에 정렬됨! 데이터 프레임 안됨)
# for i in range(len(input_split.columns)):
#     globals()["df_stwd{}".format(i)] = ["duple_list{}".format(i)]
# df_stwd0

In [ ]:
# concat 을 위한 DataFame 묶기 (o 시리즈여야 한곳에 정렬됨! 데이터 프레임 안됨)
for i in range(len(input_split.columns)):
    globals()["df_stwd{}".format(i)] = input_split["stwd{}".format(i)]
df_stwd0

In [ ]:
# df_index 묶기
for i in range(len(input_split.columns)):
    globals()["df_index{}".format(i)] = input_split["stwd{}".format(i)]
df_stwd0

In [ ]:
df_stwd = pd.concat([df_stwd, df_index], axis=1)
df_stwd

In [ ]:
pd.concat([input_split, pd.DataFrame(input_test['No'])], axis=1)


In [ ]:
# df_stwd.tail(500)

In [ ]:
list = []
for i in range(len(input_split.columns)):
    df = pd.concat([globals()["df_stwd{}".format(i)]])
df

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    df = pd.concat([df, globals()["df_stwd_df{}".format(i)]])
df 

In [ ]:
df

In [ ]:
df_dupl_cnt = pd.DataFrame()
for in in range(len(input_split.coulmns):
    df_dupl_cnt = pd.concat([df, globals()["df{}".format(i)]])

In [ ]:
dupl_list0

In [ ]:
# 컬럼 재정렬
list = []
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]
    list.append("stwd{}".format(i))
    list.append("dupl_cnt{}".format(i))
input_split = input_split[list]
input_split

In [ ]:
df_stwd = pd.DataFrame()
for i in range(len(input_split.columns)):
    df_stwd = pd.concat([df, globals()["df{}".format(i)]])
df_stwd 

In [ ]:
pd.DataFrame

In [ ]:
type(input_split["stwd{}".format(0)])

In [ ]:
a = pd.Series
a

In [ ]:
a

In [ ]:
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = input_split["stwd{}".format(i)]
    list.append('df{}'.format(i))
list

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = pd.DataFrame(input_split["stwd{}".format(i)])
#     list.append('df{}'.format(i))
# list

In [ ]:
list = []
for i in range(len(input_split.columns)):
    list.append("df{}".format(i))
list

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    df = pd.concat([df, globals()["df{}".format(i)]])
df

In [ ]:
concat([df0, df0,df0,df0,df0,df0,df0,df0,])

In [ ]:
list = ",".join(list)
list
list[5]

In [ ]:
",".join(list[0])

In [ ]:
df = pd.DataFrame()
for i in range(len(input_split.columns)):
    ccc = pd.concat(df, ",".join(list[i]))
df = pd.DataFrame()
df = pd.concat([df, df0])

In [ ]:
df = pd.DataFrame()
df = pd.concat([df, df0])

In [ ]:
",".join(list)

In [ ]:
list = ",".join(list)
list

In [ ]:
list

In [ ]:
df0

In [ ]:
list = [df0, df1, df2, df3, df4, df5]
pd.concat(list)

In [ ]:
pd.concat([df0, df1, df2, df3, df4, df5])


In [ ]:
input_split

In [ ]:
stwd_list

In [ ]:
stwd_list = input_split.columns.to_list()
for i in range(len(input_split.columns)):
    duplicated = pd.concat(stwd_list, ignore_index=True)

In [ ]:
stwd_list = input_split.columns.to_list()
pd.concat(pd.DataFrame(input_split[stwd_list]), ignore_index = True)

In [ ]:
list = []
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = input_split["stwd{}".format(i)]
    list.append(globals()["df{}".format(i)])

In [ ]:
pd.DataFrame(list)

In [ ]:
pd.DataFrame(input_split["stwd1"])

In [ ]:
input_split["stwd0"]

In [ ]:
pd.concat(pd.DataFrame(input_split["stwd0"]), pd.DataFrame(input_split["stwd1"]))

In [ ]:
df0
list

In [ ]:
globals("df0")

In [ ]:
pd.concat([df0, df1])

In [ ]:
pd.concat([df0, df1])

In [ ]:
for i in range(len(input_split.columns)):
#     pd.concat(globllist[i])
    print(list[i])

In [ ]:
for i in range(len(input_split.columns)):
#     pd.concat(["df{}".format(i)])
#     pd.concat("df{}".format(i))
    pd.concat(list)  
#     print("df{}".format(i))

In [ ]:
for i in range(len(input_split.columns)):
    print("df{}".format(i))

In [ ]:
for i in range(len(input_split.columns)):
    globals()["df{}".format(i)] = pd.DataFrame(globals()["df{}".format(i)])

In [ ]:
list = []
for i in range(len(input_split.columns)):
    list.append("df{}".format(i))

In [ ]:
df1

In [ ]:
pd.concat(list, axis=1 )
#           ignore_index=True)

In [ ]:
for i in range(len(input_split.columns)):
    pd.concat(globals()["df{}".format(i)])

In [ ]:
pd.concat([df0, df1])

In [ ]:
stwd_list = input_split.columns.to_list()
for i in range(len(input_split.columns)):
    pd.concat(pd.DataFrame(input_split["stwd{}".format(i)]))
list

In [ ]:
input_split["stwd0"]

In [ ]:
pd.DataFrame(input_split["stwd{}".format(0)])

In [ ]:
cc = pd.DataFrame(input_split["stwd{}".format(i)])

In [ ]:
aa = pd.DataFrame
for i in range(len(input_split.columns)):
    aa = aa.append(cc)
aa

In [ ]:
for i in range(len(input_split.columns)):
    a = pd.concat([input_split[list[i]]])
a

In [ ]:
a

In [ ]:
pd.concat(list)

In [ ]:
for i in range(len(input_split.columns)):
    pd.concat([globals["input_split{}".format(i)]])

In [ ]:
# 컬럼 재정렬
list = []
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]
    list.append("stwd{}".format(i))
    list.append("dupl_cnt{}".format(i))
input_split = input_split[list]
input_split

In [ ]:
input_split.columns

In [ ]:
for i in range(int(len(input_split.columns)/2)):
    aaa= pd.concat(["stwd{}".format(i)])
aaa

In [ ]:
input_split[0]

In [ ]:
input_split[0]

In [ ]:
input_split[0].count("가입")

In [ ]:
dupl_list =[]
dupl_list

In [ ]:
len(input_split.columns)

In [ ]:
input_split[0].to_list().count(input_split[0][11])

In [ ]:
globals()["stwd{}".format(i)]

In [ ]:
for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []

In [ ]:
input_split[0].to_list().count(input_split[0][3])

In [ ]:
# 성공
dupl_list = []
dupl_cnt = []


for i in range(len(input_split.columns)):
    globals()["dupl_list{}".format(i)] = []
    for j in range(len(input_split[i])):
        dupl_cnt = input_split[i].to_list().count(input_split[i][j])
        globals()["dupl_list{}".format(i)].append(dupl_cnt)
#         print(dupl_cnt)
#         print("dupl_list{}".format(i))




In [ ]:
for i in range(len(input_split.columns)):
    input_split.loc[:,"dupl_cnt{}".format(i)] = globals()["dupl_list{}".format(i)]

In [ ]:
input_split["dupl_cnt0"]

In [ ]:
# input_split
input_split[input_split["dupl_cnt0"] >= 2][["dupl_cnt0","dupl_cnt1",0]]

In [ ]:
input_split.loc[:,"dupl_cnt1"] = dupl_list1

In [ ]:
input_split

In [ ]:
for i in range(len(input_split.columns)):
    print(globals()["dupl_list{}".format(i)])

In [ ]:
a= input_split.assign(dd =  dupl_list3)
# df.insert(3,"city",['Lahore','Dehli','New York'],True)
a

In [ ]:
input_split.insert(8,"dd",dupl_list3,True)
input_split

In [ ]:
input_split[2].to_list().count('역')

In [ ]:
input_split[0].to_list()

In [ ]:
dupl_list3

In [ ]:
input_split.insert(8,"dd",dupl_list3,True)

In [ ]:
input_split

In [ ]:
pd.DataFrame(input_split[3].to_list(), dupl_list3)

In [ ]:
print(len(dupl_list))
dupl_list

In [ ]:
# type(input_split[0])
input_split[0].to_list().count('가입')

In [ ]:
# 성공
b = []
for i in range(len(input_split[0])):
    a = 0
    for j in input_split[0]:
        b.append(a)
        if input_split[0][i] == j:
            a += 1
            print(input_split[0][i] +"&&"+j+"_중복")
           
            print(a)


In [ ]:
len(b)

In [ ]:
# 성공
a = 0
for i in range(len(input_split[0])):
    for j in input_split[0]:
        if input_split[0][i] == j:
            print(input_split[0][i] +"&&"+j+"_중복")
            a += 1
            print(a)


In [ ]:
# 성공
a =0
for i in input_split[0]:
    for j in input_split[0].unique():#중복 
        if i == j:
            print(i +"&&"+j+"_중복")
            a += 1
            print(a)
#         else:
#             print(i+j)
    
    
    
#     if i == '가입':
#         print(i+"_성공")
#     else:
#         print(i)

In [ ]:
for i in input_split[0]:
    if i == '가입':
        print(i+"_성공")
    else:
        print(i)

In [ ]:
input_split[0].str == "가입"

In [ ]:
for i in a
    input_split[0] == i
    

In [ ]:
input_split[0]

In [ ]:
a = input_split[0].unique()
a

In [ ]:
len(input_split.columns)

In [ ]:
input_processing = pd.concat([input_split, input_test])
print(input_processing.columns)
input_processing.head(5)

In [ ]:
input_process["No"]

In [ ]:
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : row[10])
# # stwd1

In [ ]:
aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = ['a' ,'b', 'c', 'd','a' ,'b', 'c', 'd'])
aa

In [ ]:
aaa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist())
aaa

In [ ]:
aaa[1]

In [ ]:
aaa.columns

In [ ]:
aa['a']

In [ ]:
aa.columns

In [ ]:
for i in range(len()):
    aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = globals()["stwd{}".format(i)])
    aa

In [ ]:

try:    
    for i in range(10):
         globals()["stwd{}".format(i)] = pd.DataFrame({"stwd{}".format(i) : []})
except:
    pass
# print(stwd0)

    
        
    
# pd.DataFrame({"stwd0" : []})
        

In [ ]:
print(input_test['용어명 분리'].str.split('_', maxsplit=1))

In [ ]:
# try:
for i in range(10):
    globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_').map(lambda row : row[i])

input_test['용어명 분리'].str.split('_').map(lambda row : row[0])

In [ ]:
pd.DataFrame(input_test['용어명 분리'].str.split('_'))


In [ ]:
input_test['용어명 분리'].str.split('_').tolist() 

In [ ]:
aa = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist(), columns = ['a' ,'b', 'c', 'd','a' ,'b', 'c', 'd'])
aa

In [ ]:
input_test['용어명 분리'].str.split('_').map(lambda x:x[-1])


In [ ]:
a= pd.DataFrame(input_test['용어명 분리'].str.split('_'))
a

In [ ]:
a.str.get(0)

In [ ]:
input_test['용어명 분리'].str.split('_')[0][0]


In [ ]:
input_test['용어명 분리'].str.split('_').map(lambda row : row[-1])

In [ ]:
stwd

In [ ]:
stwd2

In [ ]:
# i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
#     globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_', maxsplit = -1).map(lambda row : row[i])

# stwd0
# stwd1

# stwd0 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[0]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd2 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd3 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd4 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd5 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd6 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd7 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd8 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd9 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
try:
    stwd10 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[10]))
except:
    pass


In [ ]:
# i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
num = 10
for i in range(10):
#     if input_test['용어명 분리'].str.split("_").map(lambda row : row[i]) 
    
    globals()["stwd{}".format(i)] = input_test['용어명 분리'].str.split('_', maxsplit = -1).map(lambda row : row[i])

stwd0
# stwd1


# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))
# stwd1 = input_test['용어명 분리'].str.split("_").map(lambda row : ''.join(row[1]))


In [ ]:
stwd1

In [ ]:
input_f = os.path.join(os.getcwd(), "input")
print(input_f)
# pd.read_excel(input_f+"test.xlsx", sheet_name="input_test", header=0)
sheet_name = "input_test"
pd.read_excel("C:/test.xlsx", sheet_name=sheet_name, header=0)

# current_path = os.path.dirname(input_f)
# input_f = os.path.join(current_path, "input")
print()
# current_path = os.path.dirname()
# print(current_path)
print(os.getcwd())

In [ ]:
print(os.getcwd())
# print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))